# Functional Programming

In Python, everything you interact with is an object. From functions to immutable data types, they are all objects. So, when I say functional programming, I mean from the perspective of not creating custom objects through classes that we define ourself. That will be covered in the next chapter.

## Operators

The first thing I want to talk about are the operators used to perform math and logic.

### Math Operators

The basic mathematical operators of Python are exactly what you expect: add (+), subtract (-), multiply (*) and divide (/). These are demonstrated below.

In [12]:
# I know the below prints aren't PEP 8, but it is cleaner for these
x = 1024                                      ;print("x = {}".format(x))
y = 2
x = y + 1                                     ;print("x = {}".format(x))
x = y - 20                                    ;print("x = {}".format(x))
x = y * 56                                    ;print("x = {}".format(x))
x = y / 2                                     ;print("x = {}".format(x))

x = 1024
x = 3
x = -18
x = 112
x = 1.0


Notice how division, even when it results in an integer, results in a float.

In addition to these basic operators, two more are provided: floor division (//) and exponentiation (*)

In [16]:
x = 4 ** 2                                    ;print("x = {}".format(x))
x = 21 // 2                                   ;print("x = {}".format(x))
x = 4.5 // 2                                  ;print("x = {}".format(x))
x = 4.0 // 2                                  ;print("x = {}".format(x))
x = 4 // 2.0                                  ;print("x = {}".format(x))

x = 16
x = 10
x = 2.0
x = 2.0
x = 2.0


Floor division clears the remainder and keeps the number in an integer if both the divisor and dividend are integers, else i

### Bitwise Operators

### Logical Operators 

### Self Application Shorthand

All symbol based operators have a shorthand form to apply the action to the current value. For example: x = x + 1 can also be written as x += 1. Below you can see many of them in action.

In [8]:
x = 1024                                      ;print("x = {}".format(x))
x += 1                                        ;print("x = {}".format(x))
x -= 10                                       ;print("x = {}".format(x))
x *= 2                                        ;print("x = {}".format(x))
x **= 3                                       ;print("x = {}".format(x))
x /= 9                                        ;print("x = {}".format(x))
x //= 2                                       ;print("x = {}".format(x))
x = int(x)  # Reset back to an integer
x &= 0b11101                                  ;print("x = {}".format(x))
x |= 0b00010                                  ;print("x = {}".format(x))
x ^= 0b10101                                  ;print("x = {}".format(x))
x >>= 2                                       ;print("x = {}".format(x))
x <<= 2                                       ;print("x = {}".format(x))

x = 1024
x = 1025
x = 1015
x = 2030
x = 8365427000
x = 929491888.8888888
x = 464745944.0
x = 24
x = 26
x = 15
x = 3
x = 12


### Order of Operations

Python follows the standard PEMDAS order of operations that you have come to expect from your calculators. However, bitwise and logical operators also need to fit in somewhere. Bitwise operators sit between exponentiation and multiplication whereas logical operators sit below everything. This is why you don't need parenthesis when doing conditionals. The complete list can be found below. Operators on the same level are evaluated left to right.
1. Parenthesis: ()
2. Exponentiation: **
3. Bitwise NOT: ~
4. Bitwise AND: &
5. Bitwise XOR: ^
6. Bitwise OR: |
7. Multiplication, Division: *,/,//
8. Addition, Subtraction: +,-
9. Logical NOT: not
10. Logical AND: and
11. Logical OR: or

## Conditionals and Looping

### If Statement

### For Loop

### While Loop

## Functions

### Standard Functions

### Generators